# Creating an agent with MCP

The [Model Context Protocol](https://www.anthropic.com/news/model-context-protocol) (MCP) introduced by Anthropic has proven to be a popular method for providing an AI agent with access to a variety of tools. [This Huggingface blog post ](https://huggingface.co/blog/Kseniase/mcp) has a nice explanation of MCP.  In this tutorial, we'll build an agent that is able to leverage MCP server provided tools.

Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[all]'

import nest_asyncio

nest_asyncio.apply()

## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the "openai" agent framework.

### Pick an LLM

Regardless of which agent framework you choose, each framework supports LiteLLM, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use OpenAI's gpt-4.1-mini.

### Pick which tools to use

 In this example, we'll add a few MCP servers that we host locally, which means we'll use a Stdio MCP server. If an MCP Server is already running and hosted elsewhere, you can use an SSE connection to access it. You can browse some of the officially supported MCP servers [here](https://github.com/modelcontextprotocol/servers/tree/main?tab=readme-ov-file).

 Lets give use two MCP servers: 
 
 * [Time](https://github.com/modelcontextprotocol/servers/tree/main/src/time): so the agent can know what time/day it is.
 * [Airbnb](https://github.com/openbnb-org/mcp-server-airbnb): so the agent can browse airbnb listings

 I will also add a custom send_message tool, that way it can ask us additional questions before getting its final answer!
  

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

In [3]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio

# This MCP Tool relies upon uvx https://docs.astral.sh/uv/getting-started/installation/
time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
)

# This MCP tool relies upon npx https://docs.npmjs.com/cli/v8/commands/npx which comes standard with npm
airbnb_tool = MCPStdio(
    command="npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)


# This is a custom tool that we will provide to the agent. For the agent to use the tool, we must provide a docstring
# and also have proper python typing for input and output parameters
def send_message(message: str) -> str:
    """Display a message to the user and wait for their response.

    Args:
        message: str
            The message to be displayed to the user.

    Returns:
        str: The response from the user.

    """
    return input(message + " ")


agent = AnyAgent.create(
    "openai",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(model_id="gpt-4.1-mini", tools=[airbnb_tool, time_tool, send_message]),
)

/Users/nbrake/scm/any-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run the Agent

Now we've configured our agent, so it's time to run it! Since it has access to airbnb listings as well as the current time, it's a perfect fit for helping me find a nice airbnb for the weekend.


In [4]:
prompt = """
I am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? Figure out the time, then ask me some questions and lets figure this out together.
"""

agent_trace = agent.run(prompt)

────────────────────────────────────────────────────── TOOL ───────────────────────────────────────────────────────

tool_name: get_current_time

input: {'timezone': 'America/New_York'}

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ {"type":"text","text":"{\n  "timezone": "America/New_York",\n  "datetime": "2025-05-21T16:01:19-04:00",\n       │
│ "is_dst": true\n}","annotations":null}                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ {"type":"text","text":"{\n  "searchUrl":                                                                        │
│ "https://www.airbnb.com/s/Ohiopyle%2C%20PA/homes?checkin=2025-05-30&checkout=2025-06-01&adults=2&children=0&inf │
│ ants=1&pets=1&price_min=301",\n  "searchResults": [\n    {\n      "id": "847793719180100758",\n      "url":     │
│ "https://www.airbnb.com/rooms/847793719180100758",\n      "demandStayListing": {\n        "id":                 │
│ "RGVtYW5kU3RheUxpc3Rpbmc6ODQ3NzkzNzE5MTgwMTAwNzU4",\n        "description": {\n          "name": {\n            │
│ "localizedStringWithTranslationPreference": "KLAE House - nestled among the trees"\n          }\n        },\n   │
│ "location": {\n          "coordinate": {\n            "latitude": 39.82369,\n            "longitude":           │
│ -79.28165\n          }\n        }\n      },\n      "badges": "Guest favorite",\n      "structuredContent": {\n  │
│ "mapCategoryInfo": "",\n        "mapSecondaryLine": "",\n        "primaryLine": "3 beds",\n                     │
│ "secondaryLine": ""\n      },\n      "avgRatingA11yLabel": "4.98 out of 5 average rating,  99 reviews",\n       │
│ "structuredDisplayPrice": {\n        "primaryLine": {\n          "accessibilityLabel": "$708 for 2 nights"\n    │
│ },\n        "explanationData": {\n          "title": "Price details",\n          "priceDetails": "$354 x 2      │
│ nights: $708"\n        }\n      }\n    },\n    {\n      "id": "1365942289098981112",\n      "url":              │
│ "https://www.airbnb.com/rooms/1365942289098981112",\n      "demandStayListing": {\n        "id":                │
│ "RGVtYW5kU3RheUxpc3Rpbmc6MTM2NTk0MjI4OTA5ODk4MTExMg==",\n        "description": {\n          "name": {\n        │
│ "localizedStringWithTranslationPreference": "Romantic Riverside Getaway! Riverview Hot Tub!"\n          }\n     │
│ },\n        "location": {\n          "coordinate": {\n            "latitude": 39.695286787565834,\n             │
│ "longitude": -79.13466864620202\n          }\n        }\n      },\n      "badges": "Superhost",\n               │
│ "structuredContent": {\n        "mapCategoryInfo": "Free cancellation",\n        "mapSecondaryLine": "",\n      │
│ "primaryLine": "",\n        "secondaryLine": "Free cancellation"\n      },\n      "avgRatingA11yLabel": "4.82   │
│ out of 5 average rating,  11 reviews",\n      "structuredDisplayPrice": {\n        "primaryLine": {\n           │
│ "accessibilityLabel": "$928 for 2 nights"\n        },\n        "explanationData": {\n          "title": "Price  │
│ details",\n          "priceDetails": "$464 x 2 nights: $928"\n        }\n      }\n    },\n    {\n      "id":    │
│ "47866422",\n      "url": "https://www.airbnb.com/rooms/47866422",\n      "demandStayListing": {\n        "id": │
│ "RGVtYW5kU3RheUxpc3Rpbmc6NDc4NjY0MjI=",\n        "description": {\n          "name": {\n                        │
│ "localizedStringWithTranslationPreference": "Luxury Cabin w/ Hot Tub adjacent to 16,000 Acres"\n          }\n   │
│ },\n        "location": {\n          "coordinate": {\n            "latitude": 39.8935,\n                        │
│ "longitude": -79.56163\n          }\n        }\n      },\n      "badges": "Superhost",\n                        │
│ "structuredContent": {\n        "mapCategoryInfo": "",\n        "mapSecondaryLine": "",\n        "primaryLine": │
│ "4 beds",\n        "secondaryLine": ""\n      },\n      "avgRatingA11yLabel": "4.87 out of 5 average rating,    │
│ 39 reviews",\n      "structuredDisplayPrice": {\n        "primaryLine": {\n          "accessibilityLabel":      │
│ "$819 for 2 nights"\n        },\n        "explanationData": {\n          "title": "Price details",\n            │
│ "priceDetails": "$410 x 2 nights: $819"\n        }\n      }\n    },\n    {\n      "id": "53218067",\n           │
│ "url": "https://www.airbnb.com/rooms/53218067",\n     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

─────────────────────────────────────────────────────── LLM ───────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ I've found a range of woodsy Airbnb cabins near Ohiopyle, PA for your stay from May 30 to June 1, 2025,         │
│ suitable for 2 adults, 1 infant, and 1 dog. Here are some options categorized by budget:                        │
│                                                                                                                 │
│ Budget up to $150 for 2 nights:                                                                                 │
│                                                                                                                 │
│  • Log Cabin 5 (Guest favorite, $211) ]8;id=658445;https://www.airbnb.com/rooms/878385397115028668\Link]8;;\                                                                      │
│  • Glamping Pod (Guest favorite, $274) ]8;id=658649;https://www.airbnb.com/rooms/1224643624417572953\Link]8;;\                                                                     │
│  • Valley View Sunrise Site @TheGreat48Escape (Guest favorite, $74) ]8;id=972355;https://www.airbnb.com/rooms/658658760148288301\Link]8;;\                                        │
│  • Nanny’s Rustic Retreat: Lil’ Red’s Nest (Superhost, $160) ]8;id=534240;https://www.airbnb.com/rooms/955289529577335421\Link]8;;\                                               │
│                                                                                                                 │
│ Moderate budget $151 - $300 for 2 nights:                                                                       │
│                                                                                                                 │
│  • Pet-friendly Quiet Cabin in the Laurel Highlands (Guest favorite, $516) ]8;id=507392;https://www.airbnb.com/rooms/661417855562950090\Link]8;;\                                 │
│  • Tall Pines (Guest favorite, $363) ]8;id=619121;https://www.airbnb.com/rooms/1141244290312390121\Link]8;;\                                                                       │
│  • Rustic Confluence Cabin with Private Stream (Rating 4.64, $365) ]8;id=252632;https://www.airbnb.com/rooms/43060311\Link]8;;\                                         │
│  • Log Cabin 8 (Superhost, $314) ]8;id=109325;https://www.airbnb.com/rooms/878385731204870228\Link]8;;\                                                                           │
│                                                                                                                 │
│ Higher budget over $300 for 2 nights:                                                                           │
│                                                                                                                 │
│  • KLAE House - nestled among the trees (Guest favorite, $708) ]8;id=214485;https://www.airbnb.com/rooms/847793719180100758\Link]8;;\                                             │
│  • Romantic Riverside Getaway! Riverview Hot Tub! (Superhost, $928) ]8;id=225930;https://www.airbnb.com/rooms/1365942289098981112\Link]8;;\                                        │
│  • Luxury Cabin w/ Hot Tub adjacent to 16,000 Acres (Superhost, $819) ]8;id=558413;https://www.airbnb.com/rooms/47866422\Link]8;;\                                      │
│  • Bright & Clean Chalet (Guest favorite, $859) ]8;id=982243;https://www.airbnb.com/rooms/53569111\Link]8;;\                                                            │
│                                                                                                                 │
│ Do any of these options catch your eye? Or would you like more options or details on any specific ones? Also,   │
│ do you have any additional preferences like number of beds, specific amenities, or proximity to certain sites?  │
╰──────────────────────────────────────────────────────

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [5]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.usage}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")

I've found a range of woodsy Airbnb cabins near Ohiopyle, PA for your stay from May 30 to June 1, 2025, suitable for 2 adults, 1 infant, and 1 dog. Here are some options categorized by budget:

Budget up to $150 for 2 nights:
- Log Cabin 5 (Guest favorite, $211) [Link](https://www.airbnb.com/rooms/878385397115028668)
- Glamping Pod (Guest favorite, $274) [Link](https://www.airbnb.com/rooms/1224643624417572953)
- Valley View Sunrise Site @TheGreat48Escape (Guest favorite, $74) [Link](https://www.airbnb.com/rooms/658658760148288301)
- Nanny’s Rustic Retreat: Lil’ Red’s Nest (Superhost, $160) [Link](https://www.airbnb.com/rooms/955289529577335421)

Moderate budget $151 - $300 for 2 nights:
- Pet-friendly Quiet Cabin in the Laurel Highlands (Guest favorite, $516) [Link](https://www.airbnb.com/rooms/661417855562950090)
- Tall Pines (Guest favorite, $363) [Link](https://www.airbnb.com/rooms/1141244290312390121)
- Rustic Confluence Cabin with Private Stream (Rating 4.64, $365) [Link](https://